In [101]:
import os
import subprocess
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME = "FongNgoo"   # ← THAY ĐỔI
REPO_NAME = "Basic_Dynamic_Prices_base_on_Demand_Model"        # ← THAY ĐỔI
BRANCH = "main"                     # hoặc "master"
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"

In [102]:
def _setup_ssh():
    """Tải SSH key từ Drive và cấu hình"""
    drive.mount('/content/drive', force_remount=False)
    SSH_DIR = '/content/drive/MyDrive/.colab_ssh'
    SSH_KEY = f"{SSH_DIR}/id_colab"
    if not os.path.exists(SSH_KEY):
        raise FileNotFoundError("SSH key chưa tồn tại! Chạy file setup trước.")

    os.makedirs('/root/.ssh', exist_ok=True)
    subprocess.run(['cp', f'{SSH_KEY}', f'{SSH_KEY}.pub', '/root/.ssh/'], check=False)
    subprocess.run(['chmod', '600', '/root/.ssh/id_colab'], check=True)
    subprocess.run(['chmod', '644', '/root/.ssh/id_colab.pub'], check=True)

    config = f"""Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
"""
    with open('/root/.ssh/config', 'w') as f:
        f.write(config)
    subprocess.run(['chmod', '600', '/root/.ssh/config'], check=True)

In [103]:
def _ensure_repo():
    """Đảm bảo repo tồn tại, clone nếu cần"""
    if not os.path.exists(REPO_DIR):
        print(f"Repo không tồn tại → Đang clone: {REPO_URL}")
        result = subprocess.run(['git', 'clone', REPO_URL, REPO_DIR])
        if result.returncode != 0:
            raise RuntimeError("Clone thất bại! Kiểm tra SSH key và repo URL.")
    # Luôn cd vào repo
    os.chdir(REPO_DIR)
    print(f"Đã vào thư mục: {REPO_DIR}")

In [106]:
def sync(message=None, auto_push=True, branch=None):
    """
    TỰ ĐỘNG SYNC 2 CHIỀU (AN TOÀN, KHÔNG LỖI getcwd)
    """
    global BRANCH
    branch = branch or BRANCH

    print("Bước 1: Thiết lập SSH...")
    _setup_ssh()

    print("Bước 2: Đảm bảo repo tồn tại...")
    _ensure_repo()

    print("Bước 3: Pull từ GitHub...")
    result = subprocess.run(['git', 'pull', 'origin', branch, '--rebase'])
    if result.returncode != 0:
        print("Pull thất bại! Kiểm tra kết nối hoặc xung đột.")
        return

    %cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
    print("Bước 4: Kiểm tra thay đổi...")
    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip().split('\n')
    status = [s for s in status if s.strip()]

    if not status:
        print("Không có thay đổi.")
        return

    print(f"Phát hiện {len(status)} thay đổi:")
    for line in status[:5]:
        print(f"   {line}")
    if len(status) > 5:
        print(f"   ... và {len(status)-5} thay đổi khác")

    if not auto_push:
        print("auto_push=False → Bỏ qua push")
        return

    msg = message or f"Auto sync from Colab - $(date)"
    print(f"Đang commit: {msg}")
    subprocess.run(['git', 'add', '.'])
    commit_result = subprocess.run(['git', 'commit', '-m', msg], capture_output=True)
    if b"nothing to commit" in commit_result.stdout:
        print("Không có gì để commit.")
        return

    print("Đang push lên GitHub...")
    push_result = subprocess.run(['git', 'push', 'origin', branch])
    if push_result.returncode == 0:
        print("SYNC HOÀN TẤT!")
    else:
        print("Push thất bại!")

In [108]:
sync()

Bước 1: Thiết lập SSH...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bước 2: Đảm bảo repo tồn tại...
Đã vào thư mục: /content/Basic_Dynamic_Prices_base_on_Demand_Model
Bước 3: Pull từ GitHub...
/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Bước 4: Kiểm tra thay đổi...
Phát hiện 2 thay đổi:
   M AUTO_SYNC_COLAB_GITHUB_SSH.ipynb
    M PPO.ipynb
Đang commit: Auto sync from Colab - $(date)
Đang push lên GitHub...
SYNC HOÀN TẤT!
